# Quickstart


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RelevanceAI/RelevanceAI-readme-docs/blob/v0.33.1/docs/GETTING_STARTED/example-applications/_notebooks/RelevanceAI_ReadMe_Quickstart_Text_Search.ipynb)


to get features such as:
- hybrid search
- multivector search
- filtered search
- etc out of the box

# What I Need

- Project & API Key (grab your API key from https://cloud.relevance.ai/ in the settings area)
- Python 3
- Relevance AI Installed as shown below. You can also visit our [Installation guide](https://docs.relevance.ai/docs)

# Installation Requirements

In [ ]:
# RelevanceAI installation
@@@ relevanceai_installation, RELEVANCEAI_SDK_VERSION=RELEVANCEAI_SDK_VERSION @@@



In [ ]:
# Vectorhub installation for quick access to Sentence Transformers
@@@vectorhub_encoders_text_tfhub_installation

# Set up Client

To use Relevance AI, a client object must be instantiated. This needs an API_key and a project name. These can be generated/access directly at https://cloud.relevance.ai/ or simply by running the cell below and following the link and the guide:


In [ ]:
@@@ client_instantiation @@@

# Vector Search

## 1) Data

For this quickstart we will be using a sample e-commerce dataset. Alternatively, you can use your own dataset for the different steps.

In [ ]:
import pandas as pd
from relevanceai.datasets import get_ecommerce_dataset

# Retrieve our sample dataset. - This comes in the form of a list of documents.
documents = get_ecommerce_dataset()

pd.DataFrame.from_dict(documents).head()

## 2) Encode

In [ ]:
from vectorhub.encoders.text.tfhub import USE2Vec
enc = USE2Vec()

# To encode the product_title field in all the documents
documents = enc.encode_documents(["product_title"], documents)

In [ ]:
documents[0].keys()

We can see that there is now a field called `product_title_use_vector_` in our data. 
`_use_vector_` is the name assigned to the model in Vectorhub and we use it when generating vectors. 
If you prefer a different name, simply modify the `__name__` attribute via
`enc.__name__ = "model_name_goes_here"`

## 3) Insert

Uploading our documents into a dataset called `quickstart-example`.
Note that each document should have a field called '_id'. Here, we generate a unique identifier using the Python uuid package.

In [ ]:
import uuid

for d in documents:
  d['_id'] = uuid.uuid4().__str__()    # Each document must have a field '_id'

# Insert the documents into the quickstart-example below.
client.insert_documents("quickstart-example", documents)

## 4) Search

Note that our dataset includes vectors generated by Universal Sentence Encoder. Therefore, in this step, we first vectorize the query using the same encoder to be able to search among the similarly generated vectors.

In [ ]:
query = "gift for my son"
query_vector = enc.encode(query)

In [ ]:
results = client.services.search.vector(
    dataset_id="quickstart-example",
    multivector_query=[
        {
            "vector": query_vector,
            "fields": ["product_title_use_vector_"] # Field to search on
        }
    ],
    page_size=5
)

You can now visit the dashboard at https://cloud.relevance.ai/sdk/search


In [ ]:
from relevanceai import show_json
show_json(
    results['results'],
    text_fields=["product_title"],
)